In [2]:
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install gradio
!pip install dashscope
!pip install modelscope_studio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

# Version 1


In [ ]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# ...


Device set to use cpu


KeyboardInterrupt: 

# Version 2

In [3]:
import time
import torch
from transformers import pipeline
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
system_message = {"role": "system", "content": ("You are a Premium Chatbot who thinks before speaking and uses 2 paragraphs: one for Reasoning and another for Response. When replying, format your answer using the following tags exactly: output your reasoning with '<|assistant reasoning|>' followed by your chain-of-thought explanation, and then output your final reply with '<|assistant response|>' followed by your answer. Also, please use lowercase for '<|system|>' and '<|user|>' tags.")}
conversation = [system_message]
def reformat_output(output_text: str) -> str:
    reformatted = output_text.replace("<System>", "<|system|>").replace("<User>", "<|user|>")
    if "<assistant>" in reformatted:
        before, assistant_block = reformatted.split("<assistant>", 1)
        if "Reasoning:" in assistant_block and "Answer:" in assistant_block:
            reasoning_part = assistant_block.split("Answer:")[0]
            response_part = assistant_block.split("Answer:")[1]
            reasoning_part = reasoning_part.replace("Reasoning:", "").strip()
            response_part = response_part.strip()
            new_assistant_text = f"<|assistant reasoning|> {reasoning_part}\n<|assistant response|> {response_part}"
            reformatted = before + new_assistant_text
        else:
            reformatted = reformatted.replace("<assistant>", "<|assistant reasoning|>")
    return reformatted
print("Type 'bye' to exit the chat.")
while True:
    user_input = input("User: ")
    if user_input.strip().lower() == "bye":
        print("Chatbot: Farewell, Sir!")
        break
    conversation.append({"role": "user", "content": user_input})
    prompt = pipe.tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    start_time = time.time()
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    end_time = time.time()
    response_time = end_time - start_time
    generated_text = outputs[0]["generated_text"]
    formatted_output = reformat_output(generated_text)
    print("Chatbot:")
    print(formatted_output)
    print(f"\nResponse Time: {response_time:.2f} seconds")
    print("\n" + "-"*50 + "\n")
    conversation.append({"role": "assistant", "content": generated_text})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cpu


Type 'bye' to exit the chat.
User: halle


KeyboardInterrupt: 

# Version 3

In [16]:
import time
import torch
import gradio as gr
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
system_message = {"role": "system", "content": ("You are a Premium Chatbot who thinks before speaking and uses 2 paragraphs: one for Reasoning and another for Response. When replying, format your answer using the following tags exactly: output your reasoning with '<|assistant reasoning|>' followed by your chain-of-thought explanation, and then output your final reply with '<|assistant response|>' followed by your answer. Also, please use lowercase for '<|system|>' and '<|user|>' tags.")}
conversation = [system_message]

def reformat_output(output_text: str) -> str:
    reformatted = output_text.replace("<System>", "<|system|>").replace("<User>", "<|user|>")
    if "<assistant>" in reformatted:
        before, assistant_block = reformatted.split("<assistant>", 1)
        if "Reasoning:" in assistant_block and "Answer:" in assistant_block:
            reasoning_part = assistant_block.split("Answer:")[0]
            response_part = assistant_block.split("Answer:")[1]
            reasoning_part = reasoning_part.replace("Reasoning:", "").strip()
            response_part = response_part.strip()
            new_assistant_text = f"<|assistant reasoning|> {reasoning_part}\n<|assistant response|> {response_part}"
            reformatted = before + new_assistant_text
        else:
            reformatted = reformatted.replace("<assistant>", "<|assistant reasoning|>")
    return reformatted

def chat(user_input, state):
    if user_input.strip().lower() == "bye":
        return "Chatbot: Farewell, Sir!", state
    state.append({"role": "user", "content": user_input})
    prompt = pipe.tokenizer.apply_chat_template(state, tokenize=False, add_generation_prompt=True)
    start_time = time.time()
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    end_time = time.time()
    response_time = end_time - start_time
    generated_text = outputs[0]["generated_text"]
    formatted_output = reformat_output(generated_text)
    state.append({"role": "assistant", "content": generated_text})
    result = f"{formatted_output}\n\nResponse Time: {response_time:.2f} seconds"
    return result, state

with gr.Blocks() as demo:
    gr.Markdown("## LogicLink Chatbot")
    chatbot_state = gr.State(value=conversation)
    chatbot_output = gr.Textbox(label="Chatbot Response", interactive=False)
    user_input_text = gr.Textbox(placeholder="Type your message here...", label="Your Message")
    send_btn = gr.Button("Send")
    send_btn.click(chat, inputs=[user_input_text, chatbot_state], outputs=[chatbot_output, chatbot_state])

demo.launch()
def generate_formatted_output(user_prompt):
    system_text = (
        "You are a Premium Chatbot who thinks before speaking and always responds "
        "in the specified format."
    )

    # Reasoning is based on the provided reasoning prompt
    assistant_reasoning = "Different ways to approach the Query : " + user_prompt + " [Insert chain-of-thought reasoning here.]"

    # Response is based on the provided response prompt (echoing the query here)
    assistant_response = user_prompt  # Replace with your actual response logic if needed

    formatted_output = (
        f"<|system|>\n{system_text}\n"
        f"<|user|>\n{user_prompt}\n"
        f"<|assistant reasoning|>\n{assistant_reasoning}\n"
        f"<|assistant response|>\n{assistant_response}"
    )

    return formatted_output

# Example usage:
user_input = "hello how are you"
output = generate_formatted_output(user_input)
print(output)


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d08e9fcb3ee401634.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<|system|>
You are a Premium Chatbot who thinks before speaking and always responds in the specified format.
<|user|>
hello how are you
<|assistant reasoning|>
Different ways to approach the Query : hello how are you [Insert chain-of-thought reasoning here.]
<|assistant response|>
hello how are you


In [ ]:
import time
import torch
import gradio as gr
from transformers import pipeline

# Initialize the text-generation pipeline
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

def logiclink_chat(user_input, history):
    if not user_input:
        return history, ""
    # Append user message
    history.append(("You", user_input))

    # Build prompts for reasoning and response
    reasoning_prompt = (
        "Reflect on all potential nuances, underlying concepts, and alternative perspectives "
        "before forming a final answer for the following query: " + user_input
    )
    response_prompt = user_input

    start = time.time()
    # Generate chain-of-thought reasoning
    reasoning_out = pipe(
        reasoning_prompt,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    reasoning = reasoning_out[0]["generated_text"].strip()

    # Generate final answer
    response_out = pipe(
        response_prompt,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    response = response_out[0]["generated_text"].strip()
    elapsed = time.time() - start

    # Append reasoning and response to history
    history.append(("LogicLink – Reasoning", reasoning))
    history.append(("LogicLink - Response", f"{response}\n\n*({elapsed:.2f}s)*"))

    return history, ""

# Build Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=3):
            gr.Markdown("## LogicLink Chat")
            chatbot = gr.Chatbot(label="Conversation", height=600)
            user_input = gr.Textbox(show_label=False, placeholder="Type your query here…")
            send = gr.Button("Send", variant="primary")
        with gr.Column(scale=1):
            gr.Markdown(
                "**How LogicLink Works**\n\n"
                "- You type a question.\n"
                "- LogicLink first shows its _chain-of-thought_.\n"
                "- Then it gives the concise answer below.\n"
                "- Conversation history is on the left."
            )

    send.click(logiclink_chat, [user_input, chatbot], [chatbot, user_input])
    user_input.submit(logiclink_chat, [user_input, chatbot], [chatbot, user_input])

demo.launch()


# Version 4

In [15]:
import time
import torch
import gradio as gr
from transformers import pipeline

# Initialize the text-generation pipeline
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

def logiclink_chat(user_input, history):
    if not user_input:
        return history, ""
    # Append user message
    history.append(("You", user_input))

    # Build reasoning prompt
    reasoning_prompt = (
        "Reflect on all potential nuances, underlying concepts, and alternative perspectives "
        "before forming a final answer for the following query: " + user_input
    )

    start = time.time()
    # Generate chain-of-thought reasoning
    reasoning = pipe(
        reasoning_prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )[0]["generated_text"].strip()

    # Generate final answer
    response = pipe(
        user_input,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )[0]["generated_text"].strip()

    elapsed = time.time() - start

    # Append reasoning and response
    history.append(("LogicLink – Reasoning", reasoning))
    history.append(("LogicLink – Response", f"{response}\n\n*({elapsed:.2f}s)*"))

    return history, ""

# Custom CSS for black background, red accents, blue borders
custom_css = """
body {
    background-color: #000;
    color: #fff;
}
.gradio-container {
    border-radius: 8px;
    padding: 1rem;
}
.gr-button, .gr-button:hover {
    background-color: #e53935 !important;  /* red buttons */
    color: #fff !important;
}
.gr-textbox, .gr-chatbot {
    border: 2px solid #1e88e5 !important;  /* blue borders */
}
.gr-textbox textarea, .gr-chatbot {
    background-color: #111;
    color: #fff;
}
.gr-row {
    margin-bottom: 1rem;
}
"""

with gr.Blocks(css=custom_css) as demo:
    with gr.Row():
        with gr.Column(scale=3):
            gr.Markdown("## 🎨 LogicLink Chatbot")
            chatbot = gr.Chatbot(label="Conversation", height=600)
            user_input = gr.Textbox(show_label=False, placeholder="Type your query here…")
            send_btn = gr.Button("Send")
        with gr.Column(scale=1):
            gr.Markdown(
                "**How LogicLink Works**\n\n"
                "- You type a question.\n"
                "- LogicLink shows its _chain-of-thought_ first.\n"
                "- Then it gives the concise answer below.\n"
                "- Chat history stays on the left."
            )

    send_btn.click(logiclink_chat, [user_input, chatbot], [chatbot, user_input])
    user_input.submit(logiclink_chat, [user_input, chatbot], [chatbot, user_input])

demo.launch()


Device set to use cpu
<ipython-input-15-4a685e344178>:86: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", height=600)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a8c1409802d141d0ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Version 5

In [17]:
import uuid
import time
import json
import gradio as gr
import torch
from transformers import pipeline
import modelscope_studio.components.antd as antd
import modelscope_studio.components.antdx as antdx
import modelscope_studio.components.base as ms
import modelscope_studio.components.pro as pro
from config import DEFAULT_LOCALE, DEFAULT_THEME, get_text, user_config, bot_config, welcome_config
from ui_components.logo import Logo
from ui_components.settings_header import SettingsHeader

# Initialize TinyLlama pipeline
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)


# Custom CSS with red/blue/black theme
css = """
:root {
    --color-red: #ff4444;
    --color-blue: #1e88e5;
    --color-black: #000000;
    --color-dark-gray: #121212;
}

.gradio-container {
    background: var(--color-black) !important;
    color: white !important;
}

/* Input styling */
.gr-textbox textarea {
    background: var(--color-dark-gray) !important;
    border: 2px solid var(--color-blue) !important;
    color: white !important;
}

/* Output (chatbot) styling */
.gr-chatbot {
    background: var(--color-dark-gray) !important;
    border: 2px solid var(--color-red) !important;
}

/* User message bubbles */
.gr-chatbot .user {
    background: var(--color-blue) !important;
    border-color: var(--color-blue) !important;
}

/* Assistant message bubbles */
.gr-chatbot .bot {
    background: var(--color-dark-gray) !important;
    border: 1px solid var(--color-red) !important;
}

/* Buttons */
.gr-button {
    background: var(--color-blue) !important;
    border-color: var(--color-blue) !important;
}

/* Thinking tooltip */
.gr-chatbot .tool {
    background: var(--color-dark-gray) !important;
    border: 1px solid var(--color-red) !important;
}
"""
class Gradio_Events:
    # Add state management for generation control
    _generating = False

    @staticmethod
    def generate_response(prompt):
        """Generate both reasoning and final response using TinyLlama"""
        try:
            # Generate reasoning
            reasoning = pipe(
                f"Reflect on all potential nuances and concepts for: {prompt}",
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
            )[0]["generated_text"].strip()

            # Generate final answer
            response = pipe(
                prompt,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
            )[0]["generated_text"].strip()

            return reasoning, response
        except Exception as e:
            return None, str(e)

    @staticmethod
    def add_message(input_value, state_value):
        if not input_value.strip():
            return gr.skip()

        if not state_value["conversation_id"]:
            random_id = str(uuid.uuid4())
            state_value["conversation_id"] = random_id
            state_value["conversation_contexts"][random_id] = {"history": []}
            state_value["conversations"].append({
                "label": input_value[:20] + ("..." if len(input_value) > 20 else ""),
                "key": random_id
            })

        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]

        history.append({
            "role": "user",
            "content": input_value,
            "key": str(uuid.uuid4())
        })

        return {
            "input": gr.update(value=""),
            "chatbot": gr.update(value=history),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def submit(state_value):
        if Gradio_Events._generating:
            return gr.skip()

        Gradio_Events._generating = True
        history = state_value["conversation_contexts"][
            state_value["conversation_id"]]["history"]

        try:
            user_input = history[-1]["content"]
            start_time = time.time()
            reasoning, answer = Gradio_Events.generate_response(user_input)

            if not answer:
                raise Exception("Failed to generate response")

            elapsed = time.time() - start_time

            assistant_message = {
                "role": "assistant",
                "content": [
                    {
                        "type": "tool",
                        "content": reasoning,
                        "options": {
                            "title": get_text("Chain of Thought", "思考链"),
                            "status": "done"
                        },
                        "copyable": False,
                        "editable": False
                    },
                    {
                        "type": "text",
                        "content": f"{answer}\n\n*({elapsed:.2f}s)*",
                        "style": {"color": "#ff4444"}
                    }
                ],
                "key": str(uuid.uuid4()),
                "loading": False,
                "status": "done"
            }

            history.append(assistant_message)

        except Exception as e:
            error_message = {
                "role": "assistant",
                "content": [{
                    "type": "text",
                    "content": f'<span style="color: #ff4444">Error: {str(e)}</span>'
                }],
                "key": str(uuid.uuid4()),
                "loading": False,
                "status": "done"
            }
            history.append(error_message)

        Gradio_Events._generating = False
        return {
            "chatbot": gr.update(value=history),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def new_chat(state_value):
        state_value["conversation_id"] = ""
        return {
            "conversations": gr.update(items=state_value["conversations"]),
            "chatbot": gr.update(value=[]),
            "state": gr.update(value=state_value)
        }

    @staticmethod
    def clear_history(state_value):
        if state_value["conversation_id"]:
            state_value["conversation_contexts"][
                state_value["conversation_id"]]["history"] = []
        return {
            "chatbot": gr.update(value=[]),
            "state": gr.update(value=state_value)
        }

with gr.Blocks(css=css, fill_width=True) as demo:
    state = gr.State({
        "conversation_contexts": {},
        "conversations": [],
        "conversation_id": "",
    })

    with ms.Application(), antdx.XProvider(
            theme=DEFAULT_THEME, locale=DEFAULT_LOCALE), ms.AutoLoading():
        with antd.Row(gutter=[20, 20], wrap=False, elem_id="chatbot"):
            # Left Column
            with antd.Col(md=dict(flex="0 0 260px", span=24, order=0),
                        span=0, order=1):
                with ms.Div(elem_classes="chatbot-conversations"):
                    with antd.Flex(vertical=True, gap="small",
                                  elem_style=dict(height="100%")):
                        Logo()

                        # New Chat Button
                        with antd.Button(
                            color="primary",
                            variant="filled",
                            block=True,
                            elem_classes="new-chat-btn"
                        ) as new_chat_btn:
                            ms.Text(get_text("New Chat", "新建对话"))
                            with ms.Slot("icon"):
                                antd.Icon("PlusOutlined")

                        # Conversations List
                        with antdx.Conversations(
                                elem_classes="chatbot-conversations-list"
                        ) as conversations:
                            with ms.Slot('menu.items'):
                                with antd.Menu.Item(
                                        label="Delete",
                                        key="delete",
                                        danger=True
                                ) as conversation_delete_menu_item:
                                    with ms.Slot("icon"):
                                        antd.Icon("DeleteOutlined")

            # Right Column
            with antd.Col(flex=1, elem_style=dict(height="100%")):
                with antd.Flex(vertical=True, gap="small",
                               elem_classes="chatbot-chat"):
                    # Chat Display
                    chatbot = pro.Chatbot(
                        elem_classes="chatbot-chat-messages",
                        height=600,
                        welcome_config=welcome_config(),
                        user_config=user_config(),
                        bot_config=bot_config()
                    )

                    # Input Area
                    with antdx.Suggestion(items=[]):
                        with ms.Slot("children"):
                            with antdx.Sender(
                                placeholder="Type your message...",
                                elem_classes="chat-input"
                            ) as input:
                                with ms.Slot("prefix"):
                                    with antd.Flex(gap=4):
                                        with antd.Button(
                                            type="text",
                                            elem_classes="clear-btn"
                                        ) as clear_btn:
                                            with ms.Slot("icon"):
                                                antd.Icon("ClearOutlined")

    # Event Handlers
    input.submit(
        fn=Gradio_Events.add_message,
        inputs=[input, state],
        outputs=[input, chatbot, state]
    ).then(
        fn=Gradio_Events.submit,
        inputs=[state],
        outputs=[chatbot, state]
    )

    new_chat_btn.click(
        fn=Gradio_Events.new_chat,
        inputs=[state],
        outputs=[conversations, chatbot, state]
    )

    clear_btn.click(
        fn=Gradio_Events.clear_history,
        inputs=[state],
        outputs=[chatbot, state]
    )

demo.queue().launch()

Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63c38bc6c023ceedb9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
import uuid
import time
import json
import gradio as gr
import torch
from transformers import pipeline
import modelscope_studio.components.antd as antd
import modelscope_studio.components.antdx as antdx
import modelscope_studio.components.base as ms
import modelscope_studio.components.pro as pro
from config import DEFAULT_LOCALE, DEFAULT_THEME, get_text, user_config, bot_config, welcome_config
from ui_components.logo import Logo
from ui_components.settings_header import SettingsHeader

# Initialize TinyLlama pipeline
try:
    pipe = pipeline(
        "text-generation",
        model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        torch_dtype=torch.float32,
        device_map="cpu"
    )
    # Set tokenizer parameters to avoid truncation
    pipe.tokenizer.pad_token_id = pipe.model.config.eos_token_id
    pipe.tokenizer.padding_side = "left"
except Exception as e:
    error_msg = f"Pipeline initialization failed: {str(e)}. Please check your environment."
    print(error_msg)
    raise Exception(error_msg)

class Gradio_Events:
    _generating = False

    @staticmethod
    def logiclink_chat(user_input, history):
        if not user_input:
            return history, "No input provided"

        # Prompt for a single, robust response
        prompt = (
            f"{user_input}"
        )

        try:
            start = time.time()
            output = pipe(
                prompt,
                max_new_tokens=512,  # As requested
                min_new_tokens=50,  # Ensure minimum length
                do_sample=True,
                temperature=0.7,  # Balanced creativity
                top_k=50,
                top_p=0.95,
                no_repeat_ngram_size=3,  # Prevent repetitive short outputs
                return_full_text=False,
                truncation=False,
                eos_token_id=pipe.model.config.eos_token_id,
                pad_token_id=pipe.model.config.eos_token_id
            )[0]["generated_text"].strip()
            elapsed = time.time() - start

            # Format output
            response = f"{output}\n\n*({elapsed:.2f}s)*"

            # Append as one output
            history.append({
                "role": "assistant",
                "content": response,
                "key": str(uuid.uuid4()),
                "avatar": None
            })

            return history, response
        except Exception as e:
            error_msg = (
                f"Generation failed: {str(e)}. "
                f"Possible causes: insufficient memory, model incompatibility, or input issues."
            )
            history.append({
                "role": "assistant",
                "content": error_msg,
                "key": str(uuid.uuid4()),
                "avatar": None
            })
            return history, error_msg

    @staticmethod
    def add_message(input_value, state_value):
        # Initialize default outputs
        input_update = gr.update(value="")
        chatbot_update = gr.update(value=state_value["conversation_contexts"].get(state_value["conversation_id"], {"history": []})["history"])
        state_update = gr.update(value=state_value)

        if not input_value.strip():
            return input_update, chatbot_update, state_update

        if not state_value["conversation_id"]:
            random_id = str(uuid.uuid4())
            state_value["conversation_id"] = random_id
            state_value["conversation_contexts"][random_id] = {"history": []}
            state_value["conversations"].append({
                "label": input_value[:20] + ("..." if len(input_value) > 20 else ""),
                "key": random_id
            })

        history = state_value["conversation_contexts"][state_value["conversation_id"]]["history"]
        history.append({
            "role": "user",
            "content": input_value,
            "key": str(uuid.uuid4()),
            "avatar": None
        })

        chatbot_update = gr.update(value=history)
        state_update = gr.update(value=state_value)

        return input_update, chatbot_update, state_update

    @staticmethod
    def submit(state_value):
        if Gradio_Events._generating:
            history = state_value["conversation_contexts"].get(state_value["conversation_id"], {"history": []})["history"]
            return (
                gr.update(value=history),
                gr.update(value=state_value),
                gr.update(value="Generation in progress, please wait...")
            )

        Gradio_Events._generating = True
        history = state_value["conversation_contexts"][state_value["conversation_id"]]["history"]

        user_input = history[-1]["content"] if history and history[-1]["role"] == "user" else ""
        if not user_input:
            Gradio_Events._generating = False
            return (
                gr.update(value=history),
                gr.update(value=state_value),
                gr.update(value="No user input provided")
            )

        history, response = Gradio_Events.logiclink_chat(user_input, history)
        state_value["conversation_contexts"][state_value["conversation_id"]]["history"] = history

        Gradio_Events._generating = False
        return (
            gr.update(value=history),
            gr.update(value=state_value),
            gr.update(value=response)
        )

    @staticmethod
    def new_chat(state_value):
        state_value["conversation_id"] = ""
        return (
            gr.update(items=state_value["conversations"]),
            gr.update(value=[]),
            gr.update(value=state_value),
            gr.update(value="")
        )

    @staticmethod
    def clear_history(state_value):
        if state_value["conversation_id"]:
            state_value["conversation_contexts"][state_value["conversation_id"]]["history"] = []
        return (
            gr.update(value=[]),
            gr.update(value=state_value),
            gr.update(value="")
        )

# Custom CSS with red/blue/black theme
css = """
:root {
    --color-red: #ff4444;
    --color-blue: #1e88e5;
    --color-black: #000000;
    --color-dark-gray: #121212;
}

.gradio-container {
    background: var(--color-black) !important;
    color: white !important;
}

/* Input styling */
.gr-textbox textarea, .ms-gr-ant-input-textarea {
    background: var(--color-dark-gray) !important;
    border: 2px solid var(--color-blue) !important;
    color: white !important;
}

/* Output (chatbot) styling */
.gr-chatbot {
    background: var(--color-dark-gray) !important;
    border: 2px solid var(--color-red) !important;
}

/* Output textbox styling */
.gr-textbox.output-textbox {
    background: var(--color-dark-gray) !important;
    border: 2px solid var(--color-red) !important;
    color: white !important;
    margin-bottom: 10px;
}

/* User message bubbles */
.gr-chatbot .user {
    background: var(--color-blue) !important;
    border-color: var(--color-blue) !important;
}

/* Assistant message bubbles */
.gr-chatbot .bot {
    background: var(--color-dark-gray) !important;
    border: 1px solid var(--color-red) !important;
}

/* Buttons */
.gr-button {
    background: var(--color-blue) !important;
    border-color: var(--color-blue) !important;
}

/* Thinking tooltip */
.gr-chatbot .tool {
    background: var(--color-dark-gray) !important;
    border: 1px solid var(--color-red) !important;
}
"""

with gr.Blocks(css=css, fill_width=True) as demo:
    state = gr.State({
        "conversation_contexts": {},
        "conversations": [],
        "conversation_id": "",
    })

    with ms.Application(), antdx.XProvider(
            theme=DEFAULT_THEME, locale=DEFAULT_LOCALE), ms.AutoLoading():
        with antd.Row(gutter=[20, 20], wrap=False, elem_id="chatbot"):
            # Left Column
            with antd.Col(md=dict(flex="0 0 260px", span=24, order=0),
                        span=0, order=1):
                with ms.Div(elem_classes="chatbot-conversations"):
                    with antd.Flex(vertical=True, gap="small",
                                  elem_style=dict(height="100%")):
                        Logo()

                        # New Chat Button
                        with antd.Button(
                            color="primary",
                            variant="filled",
                            block=True,
                            elem_classes="new-chat-btn"
                        ) as new_chat_btn:
                            ms.Text(get_text("New Chat", "新建对话"))
                            with ms.Slot("icon"):
                                antd.Icon("PlusOutlined")

                        # Conversations List
                        with antdx.Conversations(
                                elem_classes="chatbot-conversations-list"
                        ) as conversations:
                            with ms.Slot('menu.items'):
                                with antd.Menu.Item(
                                        label="Delete",
                                        key="delete",
                                        danger=True
                                ) as conversation_delete_menu_item:
                                    with ms.Slot("icon"):
                                        antd.Icon("DeleteOutlined")

            # Right Column
            with antd.Col(flex=1, elem_style=dict(height="100%")):
                with antd.Flex(vertical=True, gap="small",
                               elem_classes="chatbot-chat"):
                    # Chat Display
                    chatbot = pro.Chatbot(
                        elem_classes="chatbot-chat-messages",
                        height=600,
                        welcome_config=welcome_config(),
                        user_config=user_config(),
                        bot_config=bot_config()
                    )

                    # Output Textbox
                    output_textbox = gr.Textbox(
                        label="LogicLinkVersion5",
                        lines=1,
                        elem_classes="output-textbox",
                        interactive=True
                    )

                    # Input Area
                    with antdx.Suggestion(items=[]):
                        with ms.Slot("children"):
                            with antdx.Sender(
                                placeholder="Type your message...",
                                elem_classes="chat-input"
                            ) as input:
                                with ms.Slot("prefix"):
                                    with antd.Flex(gap=4):
                                        with antd.Button(
                                            type="text",
                                            elem_classes="clear-btn"
                                        ) as clear_btn:
                                            with ms.Slot("icon"):
                                                antd.Icon("ClearOutlined")

    # Event Handlers
    input.submit(
        fn=Gradio_Events.add_message,
        inputs=[input, state],
        outputs=[input, chatbot, state]
    ).then(
        fn=Gradio_Events.submit,
        inputs=[state],
        outputs=[chatbot, state, output_textbox]
    )

    new_chat_btn.click(
        fn=Gradio_Events.new_chat,
        inputs=[state],
        outputs=[conversations, chatbot, state, output_textbox]
    )

    clear_btn.click(
        fn=Gradio_Events.clear_history,
        inputs=[state],
        outputs=[chatbot, state, output_textbox]
    )

demo.queue().launch(share=True, debug=True)

Device set to use cpu


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a536916b3e20338300.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a536916b3e20338300.gradio.live
